In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import csv
import scipy as sp
import scipy.sparse as sps
from scipy.sparse import coo_matrix
import os
import matplotlib.pyplot as pyplot
%matplotlib inline 


## Evaluation or scoring?
eval = True

In [2]:
from RecLib.DataLoad import *
# Loading from CSV files...
UCM_age = toCoo('data_UCM_age.csv', 'user', 'age')
ICM_subclass = toCoo('data_ICM_sub_class.csv', 'item', 'subclass')
ICM_asset = toCoo('data_ICM_asset.csv', 'item', 'asset')
ICM_price = toCoo('data_ICM_price.csv', 'item', 'price')
UCM_region = toCoo('data_UCM_region.csv', 'user', 'region')
target_users = toNPArray('data_target_users_test.csv')
URM = toCoo('data_train.csv', 'user', 'item')

data_UCM_age.csv
user data, ID: min 2, max 30909, length 30317, unique 30317, repetitions 0, missig 1.91 %
age data, ID: min 1, max 10, length 30317, unique 10, repetitions 30307, missig 0.00 %

data_ICM_sub_class.csv
item data, ID: min 0, max 18494, length 18495, unique 18495, repetitions 0, missig 0.00 %
subclass data, ID: min 1, max 2010, length 18495, unique 1905, repetitions 16590, missig 5.18 %

data_ICM_asset.csv
item data, ID: min 0, max 18494, length 18490, unique 18490, repetitions 0, missig 0.02 %
asset data, ID: min 0, max 0, length 18490, unique 1, repetitions 18489, missig 0.00 %

data_ICM_price.csv
item data, ID: min 0, max 18494, length 18493, unique 18493, repetitions 0, missig 0.01 %
price data, ID: min 0, max 0, length 18493, unique 1, repetitions 18492, missig 0.00 %

data_UCM_region.csv
user data, ID: min 0, max 30910, length 26609, unique 26375, repetitions 234, missig 14.67 %
region data, ID: min 0, max 7, length 26609, unique 7, repetitions 26602, missig 0.00 %


In [6]:
URM = URM.tocsr()

In [7]:
## Split dataset (train % of .9999 gives similar performance on test set and competition set)
from Notebooks_utils.data_splitter import train_test_holdout
URM_train, URM_test = train_test_holdout(URM, train_perc = 0.8)


if not eval:
    URM_train = URM
else:
    from Base.Evaluation.Evaluator import EvaluatorHoldout
    evaluator_validation = EvaluatorHoldout(URM_test, cutoff_list=[10])

In [8]:
## SLIM BPR Recommender - MAP 0.41 on test set
## set True if you want to train and compile SLIM BPR Recommender
if False:
    from SLIM_BPR.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython 
    slim_rec = SLIM_BPR_Cython(URM_train, recompile_cython=False, verbose = False)
    MAP_LIST = []
    epochsList = [300]
    batchSize = [50]
    tklist = [10]
    lrs = [1e-3]

    for epochsN in epochsList:
        for bs in batchSize:
            for tk in tklist:
                for lr in lrs:
                    do_not_display_hystory = slim_rec.fit(epochs=epochsN, batch_size=bs, sgd_mode='adagrad', learning_rate=lr, topK = tk)
                    if eval:
                        dict_scores = (evaluator_validation.evaluateRecommender(slim_rec))[0][10]
                        MAP_LIST.append(('epoch, batch, topK, lr :' + str(epochsN) + ' ' + str(bs)+ ' ' + str(tk) + ' ' + str(lr), dict_scores['MAP']))
    if eval:
        MAP_LIST

Unable to read memory status: list index out of range
SLIM_BPR_Recommender: Epoch 1 of 300. Elapsed time 0.29 sec
SLIM_BPR_Recommender: Epoch 2 of 300. Elapsed time 0.52 sec
SLIM_BPR_Recommender: Epoch 3 of 300. Elapsed time 0.63 sec
SLIM_BPR_Recommender: Epoch 4 of 300. Elapsed time 0.73 sec
SLIM_BPR_Recommender: Epoch 5 of 300. Elapsed time 0.81 sec
SLIM_BPR_Recommender: Epoch 6 of 300. Elapsed time 0.89 sec
SLIM_BPR_Recommender: Epoch 7 of 300. Elapsed time 0.97 sec
SLIM_BPR_Recommender: Epoch 8 of 300. Elapsed time 1.05 sec
SLIM_BPR_Recommender: Epoch 9 of 300. Elapsed time 1.14 sec
SLIM_BPR_Recommender: Epoch 10 of 300. Elapsed time 1.23 sec
SLIM_BPR_Recommender: Epoch 11 of 300. Elapsed time 1.32 sec
SLIM_BPR_Recommender: Epoch 12 of 300. Elapsed time 1.41 sec
SLIM_BPR_Recommender: Epoch 13 of 300. Elapsed time 1.49 sec
SLIM_BPR_Recommender: Epoch 14 of 300. Elapsed time 1.56 sec
SLIM_BPR_Recommender: Epoch 15 of 300. Elapsed time 1.64 sec
SLIM_BPR_Recommender: Epoch 16 of 300. E

SLIM_BPR_Recommender: Epoch 134 of 300. Elapsed time 11.36 sec
SLIM_BPR_Recommender: Epoch 135 of 300. Elapsed time 11.42 sec
SLIM_BPR_Recommender: Epoch 136 of 300. Elapsed time 11.48 sec
SLIM_BPR_Recommender: Epoch 137 of 300. Elapsed time 11.55 sec
SLIM_BPR_Recommender: Epoch 138 of 300. Elapsed time 11.62 sec
SLIM_BPR_Recommender: Epoch 139 of 300. Elapsed time 11.67 sec
SLIM_BPR_Recommender: Epoch 140 of 300. Elapsed time 11.73 sec
SLIM_BPR_Recommender: Epoch 141 of 300. Elapsed time 11.79 sec
SLIM_BPR_Recommender: Epoch 142 of 300. Elapsed time 11.88 sec
SLIM_BPR_Recommender: Epoch 143 of 300. Elapsed time 11.98 sec
SLIM_BPR_Recommender: Epoch 144 of 300. Elapsed time 12.07 sec
SLIM_BPR_Recommender: Epoch 145 of 300. Elapsed time 12.16 sec
SLIM_BPR_Recommender: Epoch 146 of 300. Elapsed time 12.25 sec
SLIM_BPR_Recommender: Epoch 147 of 300. Elapsed time 12.34 sec
SLIM_BPR_Recommender: Epoch 148 of 300. Elapsed time 12.42 sec
SLIM_BPR_Recommender: Epoch 149 of 300. Elapsed time 12

SLIM_BPR_Recommender: Epoch 265 of 300. Elapsed time 21.01 sec
SLIM_BPR_Recommender: Epoch 266 of 300. Elapsed time 21.07 sec
SLIM_BPR_Recommender: Epoch 267 of 300. Elapsed time 21.13 sec
SLIM_BPR_Recommender: Epoch 268 of 300. Elapsed time 21.20 sec
SLIM_BPR_Recommender: Epoch 269 of 300. Elapsed time 21.26 sec
SLIM_BPR_Recommender: Epoch 270 of 300. Elapsed time 21.32 sec
SLIM_BPR_Recommender: Epoch 271 of 300. Elapsed time 21.39 sec
SLIM_BPR_Recommender: Epoch 272 of 300. Elapsed time 21.45 sec
SLIM_BPR_Recommender: Epoch 273 of 300. Elapsed time 21.51 sec
SLIM_BPR_Recommender: Epoch 274 of 300. Elapsed time 21.57 sec
SLIM_BPR_Recommender: Epoch 275 of 300. Elapsed time 21.64 sec
SLIM_BPR_Recommender: Epoch 276 of 300. Elapsed time 21.70 sec
SLIM_BPR_Recommender: Epoch 277 of 300. Elapsed time 21.77 sec
SLIM_BPR_Recommender: Epoch 278 of 300. Elapsed time 21.84 sec
SLIM_BPR_Recommender: Epoch 279 of 300. Elapsed time 21.90 sec
SLIM_BPR_Recommender: Epoch 280 of 300. Elapsed time 21

[('epoch, batch, topK, lr :300 50 10 0.001', 0.04076072648768307)]

In [9]:
## ItemCFKNNRecommender - MAP 0.47 on test set
# If you want to compile and train KNN set to True
if True:
    from KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
    itemKNN = ItemKNNCFRecommender(URM_test)
    itemKNN.fit(shrink=8, topK=12)


    if eval:
        evaluator_validation.evaluateRecommender(itemKNN)


ItemKNNCFRecommender: URM Detected 3656 (11.83 %) cold users.
ItemKNNCFRecommender: URM Detected 3218 (17.40 %) cold items.
Similarity column 18495 ( 100 % ), 6851.49 column/sec, elapsed time 0.04 min
EvaluatorHoldout: Processed 27255 ( 100.00% ) in 15.26 sec. Users per second: 1786


({10: {'ROC_AUC': 0.0,
   'PRECISION': 0.0,
   'PRECISION_RECALL_MIN_DEN': 0.0,
   'RECALL': 0.0,
   'MAP': 0.0,
   'MRR': 0.0,
   'NDCG': 0.0,
   'F1': 0.0,
   'HIT_RATE': 0.0,
   'ARHR': 0.0,
   'RMSE': nan,
   'NOVELTY': 0.006814142915981868,
   'AVERAGE_POPULARITY': 0.2199428052409083,
   'DIVERSITY_MEAN_INTER_LIST': 0.9892951940984678,
   'DIVERSITY_HERFINDAHL': 0.9989258896347812,
   'COVERAGE_ITEM': 0.7105163557718303,
   'COVERAGE_USER': 0.881724952282359,
   'DIVERSITY_GINI': 0.22912861533283296,
   'SHANNON_ENTROPY': 11.409584874727297}},
 'CUTOFF: 10 - ROC_AUC: 0.0000000, PRECISION: 0.0000000, PRECISION_RECALL_MIN_DEN: 0.0000000, RECALL: 0.0000000, MAP: 0.0000000, MRR: 0.0000000, NDCG: 0.0000000, F1: 0.0000000, HIT_RATE: 0.0000000, ARHR: 0.0000000, RMSE: nan, NOVELTY: 0.0068141, AVERAGE_POPULARITY: 0.2199428, DIVERSITY_MEAN_INTER_LIST: 0.9892952, DIVERSITY_HERFINDAHL: 0.9989259, COVERAGE_ITEM: 0.7105164, COVERAGE_USER: 0.8817250, DIVERSITY_GINI: 0.2291286, SHANNON_ENTROPY: 1

In [11]:
## HYBRID: KNN + SLIM BPR
# Set to True if you want to compile and train this hybrid rec
from RecLib.ItemKNNScoresHybridRecommender import *
if False: 
    hybridrecommender = ItemKNNScoresHybridRecommender(URM_test, itemKNN, slim_rec)
    hybridrecommender.fit(alpha = 0.5)
    
    if eval:
        evaluator_validation.evaluateRecommender(hybridrecommender)

ItemKNNScoresHybridRecommender: URM Detected 3656 (11.83 %) cold users.
ItemKNNScoresHybridRecommender: URM Detected 3218 (17.40 %) cold items.


NameError: name 'check_matrix' is not defined

In [ ]:
## TEST & VISUALIZE DIFFERENT RECSYS PERFORMANCE BASED ON N° OF INTERACTIONS HELD BY USERS
# Please, fill empty list with pretrained recommenders you want to compare
recommendersToCompare = []
if eval:
    from RecLib.PerUserGroupTest import *
    compare(URM_train, URM_test, recommendersToCompare)

In [12]:
## Predict
## SET WHICH RECOMMENDER YOU WANNA USE
final_Rec = itemKNN
if not eval:
    output = []
    for user_id in target_users:
        output.append((user_id, final_Rec.recommend(user_id, cutoff=10)))

In [13]:
#Writedown results
if not eval:
    with open('submission.csv', 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["user_id", "item_list"])
        for row in output:
          ranking = ''
          for val in row[1]:
            ranking = ranking + str(val) + ' '
          writer.writerow([row[0], ranking[:-1]])